In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('data/Life_Expectancy_Data.csv')

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")

In [ ]:
le.classes_

# TEST

In [10]:
# Load your dataset into a DataFrame
import ppscore as pps
data = pd.read_csv('data/car_price_dataset.csv')

# Separate features (X) and target variable (y)
X = data.drop('selling_price', axis=1)
y = data['selling_price']


In [11]:
# Calculate PPS matrix
pps_matrix = pps.matrix(X)

In [15]:
# Assuming X is your feature DataFrame
numeric_features = X.select_dtypes(include=['name'])  # Select only numeric columns

# Now you can use the numeric_features DataFrame for further operations


TypeError: data type 'name' not understood

In [14]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(pps_matrix, cmap="Blues", annot=True)
plt.title("Predictive Power Score Matrix")
plt.show()


ValueError: could not convert string to float: 'name'

<Figure size 1000x800 with 0 Axes>

In [ ]:
# Set a threshold for PPS score
threshold = 0.3

# Select features with PPS score above the threshold
selected_features = pps_matrix[pps_matrix['ppscore'] > threshold]['x'].tolist()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X[selected_features], y, test_size=0.2, random_state=42)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")
